# Processing ERA5
You will need precipitation, SSTs and temperatures.
Ideally temperatures from ERA5.1 and ERA5, although differences in the troposphere are small

Data can be downloaded here: https://confluence.ecmwf.int/display/CKB/How+to+download+ERA5

In [1]:
from dask.distributed import Client
import multiprocessing

ncpu = multiprocessing.cpu_count()
threads = 8
nworker = ncpu // threads
print(
    f"Number of CPUs: {ncpu}, number of threads: {threads}, number of workers: {nworker}"
)

Number of CPUs: 48, number of threads: 8, number of workers: 6


In [2]:
client = Client(
    processes=True, threads_per_worker=threads, n_workers=nworker, memory_limit="256GB"
)
client

Client Scheduler: tcp://127.0.0.1:41388 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 6 Cores: 48 Memory: 314.57 GB


In [3]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import metcalc
import scipy.stats as stat
import metpy
import pandas as pd

In [4]:
SST_mask = xr.open_dataset("../data/sea_mask_icon.nc")

In [5]:
def calc_PRSST(ds,lat_bound) :
   
    ds['SST'] = ds.ts * SST_mask.sea
    ds_tropics = ds.sel(lat=slice(lat_bound,-lat_bound))
    
    PRSST = metcalc.xr_fldmean(ds_tropics.pr * ds_tropics.SST) / metcalc.xr_fldmean( ds_tropics.pr * SST_mask.sea )
    
    return PRSST.to_dataset(name='PRSST')
    


For the PRSST it should be sufficient to use the standard ERA5 precip and SST, because ERA5.1 only relly shows differences in upper troposphere and stratosphere.

## Read

In [6]:
era5_pr = xr.open_dataset("../data/era5/era5_2dvars_mm_1979-2014.nc")
era5_sst = xr.open_dataset("../data/era5/era5_SST_mm_1979-2014.nc")
era5_ta = xr.open_dataset("../data/era5/era5_T_mm_1979-2014.nc")
era51_ta = xr.open_dataset("../data/era5/era5.1_T_mm_2000-2006.nc")

lat_bound = 20

## Calculate tropical mean

In [16]:
era_5_ta_tropical_mean = metcalc.xr_fldmean(era5_ta.sel(lat=slice(lat_bound,-lat_bound)))
era_51_ta_tropical_mean = metcalc.xr_fldmean(era51_ta.drop("time_bnds").sel(lat=slice(lat_bound,-lat_bound)))

In [17]:
era_5_ta_tropical_mean.to_netcdf(path='../data/era5/era5_T_mm_tropicalmean_' + lat_bound +'lat_1979-2014.nc' , mode='w')
era_51_ta_tropical_mean.to_netcdf(path='../data/era5/era5.1_T_mm_tropicalmean_' + lat_bound +'lat_1979-2014.nc' , mode='w')

## Calculate PRSST

In [147]:
era5_mask = xr.where( era5_sst.isel(time=0).SSTK.isnull() , era5_sst.isel(time=0).SSTK, 1 )

In [148]:
era_tropics = era5_sst.sel(lat=slice(lat_bound,-lat_bound))
era_tropics["pr"] = era5_pr.sel(lat=slice(lat_bound,-lat_bound)).TP

PRSST_era5 = metcalc.xr_fldmean(era_tropics.pr * era_tropics.SSTK) / metcalc.xr_fldmean( era_tropics.pr * era5_mask.sel(lat=slice(lat_bound,-lat_bound)) )


In [162]:
PRSST_era5_ds = PRSST_era5.to_dataset(name="PRSST")
PRSST_era5_ds.to_netcdf('../data/era5/PRSST_era5_20.nc',)

The calculation can be done with setting the land values to zero, or to NaN, if the corresponding ocean mask is then also zero or NaN over land. By using NaN the fieldmean will not take the NaN into account, unlike in the case for 0 over land. However, by normalising by the precipitation in the denominator these differences do not matter, as long as the land mask in the denominator uses the same values over land as the SST data (zeros or NaNs).

Thats why we can use both calcualtions and chose whichever is more convenient to us.

A quick demonstration: Above we have used NaNs over land, here we do zeros:

In the end, the difference between the datasets is zero

In [151]:
a = xr.where( era5_sst.isel(time=0).SSTK > 0 , era5_sst.isel(time=1).SSTK, 0 )
era5_mask_2 = xr.where( a < 100, a, 1 )

era5_sst["SST"] = xr.where( era5_sst.SSTK > 0 , era5_sst.SSTK, 0 )

In [154]:
era_tropics = era5_sst.sel(lat=slice(lat_bound,-lat_bound))
era_tropics["pr"] = era5_pr.sel(lat=slice(lat_bound,-lat_bound)).TP

PRSST_era5_2 = metcalc.xr_fldmean(era_tropics.pr * era_tropics.SST) / metcalc.xr_fldmean( era_tropics.pr * era5_mask_2.sel(lat=slice(lat_bound,-lat_bound)) )


In [155]:
PRSST_era5 - PRSST_era5_2

<xarray.DataArray (time: 432)>
array([ 5.68434189e-14, -5.68434189e-14, -5.68434189e-14,  0.00000000e+00,
        5.68434189e-14, -5.68434189e-14,  0.00000000e+00,  0.00000000e+00,
       -5.68434189e-14,  0.00000000e+00, -5.68434189e-14,  0.00000000e+00,
       -5.68434189e-14, -5.68434189e-14,  0.00000000e+00,  5.68434189e-14,
        0.00000000e+00,  0.00000000e+00,  5.68434189e-14,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -5.68434189e-14, -5.68434189e-14,
       -5.68434189e-14,  0.00000000e+00, -5.68434189e-14,  0.00000000e+00,
        0.00000000e+00, -5.68434189e-14,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -5.68434189e-14,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  5.68434189e-14,
        5.68434189e-14,  0.00000000e+00,  0.00000000e+00, -5.68434189e-14,
        0.00000000e+00, -5.68434189e-14, -5.68434189e-14,  5.68434189e-14,
        0.00000000e+00,  0.00000000e+00, -5.68434189e-14, -5.68434189e-14,
        0.00000000e+00,  5.68434189e-14,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00, -5.68434189e-14,  5.68434189e-14,  0.00000000e+00,
       -5.68434189e-14,  0.00000000e+00,  5.68434189e-14,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  5.68434189e-14, -5.68434189e-14,
        0.00000000e+00,  0.00000000e+00, -5.68434189e-14, -5.68434189e-14,
       -5.68434189e-14,  5.68434189e-14,  0.00000000e+00,  5.68434189e-14,
       -5.68434189e-14, -5.68434189e-14, -5.68434189e-14,  0.00000000e+00,
...
        0.00000000e+00,  0.00000000e+00,  5.68434189e-14, -5.68434189e-14,
        0.00000000e+00,  5.68434189e-14,  5.68434189e-14,  0.00000000e+00,
        5.68434189e-14,  0.00000000e+00,  0.00000000e+00,  5.68434189e-14,
        0.00000000e+00, -5.68434189e-14,  0.00000000e+00, -5.68434189e-14,
       -5.68434189e-14,  0.00000000e+00,  0.00000000e+00, -5.68434189e-14,
        5.68434189e-14,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        5.68434189e-14, -5.68434189e-14,  0.00000000e+00,  5.68434189e-14,
        0.00000000e+00,  0.00000000e+00,  5.68434189e-14,  0.00000000e+00,
       -5.68434189e-14,  0.00000000e+00,  5.68434189e-14,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00, -5.68434189e-14,  0.00000000e+00,
       -5.68434189e-14,  5.68434189e-14,  5.68434189e-14,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -5.68434189e-14,  0.00000000e+00, -5.68434189e-14,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        5.68434189e-14,  0.00000000e+00, -5.68434189e-14,  0.00000000e+00,
       -5.68434189e-14, -5.68434189e-14,  5.68434189e-14, -5.68434189e-14,
        5.68434189e-14,  5.68434189e-14,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  5.68434189e-14, -5.68434189e-14,
        5.68434189e-14,  0.00000000e+00, -5.68434189e-14,  0.00000000e+00])
Coordinates:
  * time     (time) datetime64[ns] 1979-01-01 1979-02-01 ... 2014-12-01